In [1]:
import theano.tensor as T
from theano import function
x=T.dmatrix('x')
s=1/(1+T.exp(-x))
logistic=function([x],s)
logistic([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [2]:
a,b=T.dmatrices('a','b')
diff=a-b
abs_diff=abs(diff)
diff_squared=diff**2
f=function([a,b],[diff,abs_diff,diff_squared])

In [3]:
f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

In [4]:
from theano import In
x, y = T.dscalars('x', 'y')
z = x + y
f=function([x,In(y,value=1)],z)

In [26]:
from theano import shared
state=shared(0)
inc=T.iscalar('inc')
f=function([inc],state,updates=[(state,state+inc)],on_unused_input='ignore')

In [6]:
print state.get_value()

0


In [7]:
f(1)

array(0)

In [8]:
print state.get_value()

1


In [10]:
f2=function([inc],state,updates=[(state,state-inc)])

In [11]:
f2(1)

array(1)

In [12]:
state

<TensorType(int64, scalar)>

In [13]:
state.get_value()

array(0)

In [16]:
func_state=state*2+inc
foo=T.scalar(dtype=state.dtype)
func=function([inc,foo],func_state,givens=[(state,foo)])

In [17]:
func(1,3)

array(7)

In [20]:
new_state=shared(0)
new_f=f.copy(swap={state:new_state})

In [21]:
new_f(100)

[array(0)]

In [22]:
state.get_value()

array(0)

In [23]:
new_state.get_value()

array(100)

In [27]:
null_f=f.copy(delete_updates=True)

In [28]:
null_f(10)

[array(0)]

In [29]:
staet

NameError: name 'staet' is not defined

In [31]:
state.get_value()

array(0)

In [32]:
from theano.tensor.shared_randomstreams import RandomStreams

In [33]:
srng=RandomStreams(seed=234)

In [34]:
print srng

In [35]:
rv_u=srng.uniform((2,2))

In [36]:
rv_n=srng.uniform((2,2))

In [37]:
f=function([],rv_u)
g=function([],rv_n,no_default_updates=True)

In [38]:
nearly_zeroes=function([],rv_u+rv_u-2*rv_u)

## Seeding Streams

In [39]:
rng_value=rv_u.rng.get_value(borrow=True)

In [40]:
rng_value.seed(89234)

In [41]:
rv_u.rng.set_value(rng_value, borrow=True)

In [42]:
#Seeding all RVs allocated by a Random Stream object
srng.seed(2455)

## Sharing Streams between functions

In [45]:
state_after_v0 = rv_u.rng.get_value().get_state()
nearly_zeroes()

array([[ 0.,  0.],
       [ 0.,  0.]])

In [46]:
v1=f()

In [48]:
rng_value=rv_u.rng.get_value(borrow=True)
rng_value.set_state(state_after_v0)
rv_u.rng.set_value(rng_value, borrow=True)
v2=f() #v2!=v1
v3=f() #v3==v1

## Copying Random States between Theano Graphs

In [49]:
from __future__ import print_function

In [51]:
import theano
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams

In [52]:
class Graph():
    def __init__(self,seed=123):
        self.rng=RandomStreams(seed)
        self.y=self.rng.uniform(size=(1,))

In [53]:
g1=Graph()
f1=function([],g1.y)

In [54]:
g2=Graph(seed=987)
f2=function([],g2.y)

In [55]:
f1()

array([ 0.72803009])

In [56]:
f2()

array([ 0.55056769])

In [61]:
def copy_random_state(g1,g2):
    if isinstance(g1.rng,MRG_RandomStreams):
        g2.rng.rstate=g1.rng.rstate
    for(su1,su2) in zip(g1.rng.state_updates,g2.rng.state_updates):
        su2[0].set_value(su1[0].get_value())

In [62]:
copy_random_state(g1,g2)

In [63]:
f1()

array([ 0.59044123])

In [64]:
f2()

array([ 0.59044123])